# Re Run Step Function Executions

### Imports & Creds :

In [15]:
import boto3
import datetime
from dateutil import tz
import sys

# Set up boto3 client for Step Functions
stepfunctions = boto3.client('stepfunctions')

# Constants
STATE_MACHINE_NAME = "RAPISAMStateMachine-1YwcU3XmIcZ5"
SUPPLIER_ID = "MACKIE"
START_TIME_UTC = datetime.datetime(2025, 5, 27, 14, 0, 0, tzinfo=datetime.timezone.utc)
NOW_UTC = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc)

### Functons :

In [16]:
def get_state_machine_arn(name):
    print(f"[INFO] Looking up state machine ARN for '{name}'...")
    paginator = stepfunctions.get_paginator('list_state_machines')
    for page in paginator.paginate():
        for sm in page['stateMachines']:
            if sm['name'] == name:
                print(f"[INFO] Found ARN: {sm['stateMachineArn']}")
                return sm['stateMachineArn']
    raise ValueError(f"State machine '{name}' not found.")

def list_supplier_executions(state_machine_arn, supplier_id, start_time, end_time):
    print(f"[INFO] Listing executions for supplier '{supplier_id}' between {start_time} and {end_time}...")

    executions = []
    paginator = stepfunctions.get_paginator('list_executions')
    total_scanned = 0
    matched_count = 0

    for page in paginator.paginate(stateMachineArn=state_machine_arn):
        for exe in page['executions']:
            total_scanned += 1

            # Stop if execution is older than the start time (pagination is in DESC order)
            if exe['startDate'] < start_time:
                print("\n[INFO] Reached older executions. Stopping pagination.")
                print(f"[INFO] Total executions scanned: {total_scanned}")
                print(f"[INFO] Matching executions found: {matched_count}")
                return executions

            # Show live scan progress on one line
            sys.stdout.write(f"\r[SCAN] Scanned: {total_scanned} | Matches: {matched_count}")
            sys.stdout.flush()

            if supplier_id in exe['name'] and start_time <= exe['startDate'] <= end_time:
                matched_count += 1
                executions.append(exe)

    print()  # newline after progress bar
    print(f"[INFO] Total executions scanned: {total_scanned}")
    print(f"[INFO] Matching executions found: {matched_count}")
    return executions


def get_execution_input(execution_arn):
    print(f"[INFO] Fetching input for execution ARN: {execution_arn}")
    response = stepfunctions.describe_execution(executionArn=execution_arn)
    return response['input']

def start_retry_executions(executions, state_machine_arn):
    print(f"[INFO] Retrying {len(executions)} executions...")

    for i, exe in enumerate(executions, 1):
        new_name = f"{exe['name']}-Retry"
        print(f"[{i}/{len(executions)}] Preparing to retry: {new_name}")

        try:
            print(f"[{i}] Fetching input for {exe['executionArn']}...")
            input_data = get_execution_input(exe['executionArn'])
            print(f"[{i}] Input fetched. Starting retry...")

            stepfunctions.start_execution(
                stateMachineArn=state_machine_arn,
                name=new_name[:80],  # Trim if over AWS limit
                input=input_data
            )

            print(f"[{i}] ✅ Retry started: {new_name}")

        except stepfunctions.exceptions.ExecutionAlreadyExists as e:
            print(f"[{i}] ⚠️ Skipped (already exists): {new_name}")
        except Exception as e:
            print(f"[{i}] ❌ Error starting execution {new_name}: {e}")



### Re Run Executions

In [17]:
def main():
    print("[INFO] Script started.")
    try:
        state_machine_arn = get_state_machine_arn(STATE_MACHINE_NAME)
        executions = list_supplier_executions(
            state_machine_arn, SUPPLIER_ID, START_TIME_UTC, NOW_UTC
        )

        if not executions:
            print("[INFO] No matching executions found. Exiting.")
            return

        start_retry_executions(executions, state_machine_arn)
    except Exception as e:
        print(f"[ERROR] {e}")

    print("[INFO] Script finished.")

if __name__ == "__main__":
    main()

[INFO] Script started.
[INFO] Looking up state machine ARN for 'RAPISAMStateMachine-1YwcU3XmIcZ5'...
[INFO] Found ARN: arn:aws:states:us-east-1:389633136494:stateMachine:RAPISAMStateMachine-1YwcU3XmIcZ5
[INFO] Listing executions for supplier 'MACKIE' between 2025-05-27 14:00:00+00:00 and 2025-05-28 13:47:50.630194+00:00...
[SCAN] Scanned: 5736 | Matches: 9
[INFO] Reached older executions. Stopping pagination.
[INFO] Total executions scanned: 5737
[INFO] Matching executions found: 9
[INFO] Retrying 9 executions...
[1/9] Preparing to retry: MACKIE_005536_251482815380696_20250528133813.7036-Retry
[1] Fetching input for arn:aws:states:us-east-1:389633136494:execution:RAPISAMStateMachine-1YwcU3XmIcZ5:MACKIE_005536_251482815380696_20250528133813.7036...
[INFO] Fetching input for execution ARN: arn:aws:states:us-east-1:389633136494:execution:RAPISAMStateMachine-1YwcU3XmIcZ5:MACKIE_005536_251482815380696_20250528133813.7036
[1] Input fetched. Starting retry...
[1] ✅ Retry started: MACKIE_00553